In [1]:
import pickle
import os
import pandas as pd
from matplotlib import pyplot as plt
os.chdir(os.path.expanduser('~/wcEcoli/out/'))
# noinspection PyUnresolvedReferences
import numpy as np
import plotly.graph_objects as go
from models.ecoli.analysis import cohortAnalysisPlot
from wholecell.analysis.analysis_tools import (exportFigure,
	read_bulk_molecule_counts, read_stacked_bulk_molecules, read_stacked_columns)
from wholecell.io.tablereader import TableReader
from sklearn.metrics import r2_score

In [9]:
# CLNE 
VS_log_CLNE = pd.read_csv('~/wcEcoli/models/ecoli/analysis/local_notebooks/C_limited_PDR_analyses_with_published_paper_data/previous_models/CLNE_files/cohort_saved_protein_count_data_10_09/validation_data/log_data/Log10_Schmidt_Comparison_startGen_2.csv')
VW_log_CLNE = pd.read_csv('~/wcEcoli/models/ecoli/analysis/local_notebooks/C_limited_PDR_analyses_with_published_paper_data/previous_models/CLNE_files/cohort_saved_protein_count_data_10_09/validation_data/log_data/Log10_Wisniewski_Comparison_startGen_2.csv')

# CLClimNE
# TODO: CHANGE THESE TO THE CORRECT PATH
VS_log_CLClimNE = pd.read_csv(
    '~/wcEcoli/models/ecoli/analysis/local_notebooks/C_limited_PDR_analyses_with_published_paper_data/Clim_sorting_combos/Clim3/Clim3d/cohort_saved_protein_count_data/validation_data/log_data/Log10_Schmidt_Comparison_startGen_2.csv')
VW_log_CLClimNE = pd.read_csv(
    '~/wcEcoli/models/ecoli/analysis/local_notebooks/C_limited_PDR_analyses_with_published_paper_data/Clim_sorting_combos/Clim3/Clim3d/cohort_saved_protein_count_data/validation_data/log_data/Log10_Wisniewski_Comparison_startGen_2.csv')

In [2]:
# specifiy the current sequence being used: 
current_sequence = 'CLClim3dNE'
Clim_name = 'Clim3d'

In [17]:
# load in the protease file: 
priority_proteases = pd.read_csv('~/wcEcoli/models/ecoli/analysis/local_notebooks/C_limited_PDR_analyses_with_published_paper_data/supplementary_data/priority_protease_assignments_0.tsv',  skiprows=1, delimiter='\t')

# load in the table from nora's parca analysis script (grabbed from the other branch) 
CLClimNE_HLs = pd.read_csv('~/wcEcoli/models/ecoli/analysis/local_notebooks/C_limited_PDR_analyses_with_published_paper_data/Clim_sorting_combos/Clim3/Clim3d/protein_half_lives_Clim3d.tsv', skiprows=0, delimiter='\t')

In [18]:
# get the gene symbols
def get_gene_symbols_for_monomer_ids():
    # code adapted from convert_to_flat.py
    import io
    from wholecell.io import tsv
    from wholecell.utils.filepath import ROOT_PATH
    RNAS_FILE = os.path.join(ROOT_PATH, 'reconstruction', 'ecoli', 'flat', 'rnas.tsv')
    with io.open(RNAS_FILE, 'rb') as f:
        reader = tsv.reader(f, delimiter='\t')
        headers = next(reader)
        while headers[0].startswith('#'):
            headers = next(reader)

        gene_symbol_index = headers.index('common_name')
        protein_id_index = headers.index('monomer_ids')
        monomer_ids_to_gene_symbols = {}
        for line in reader:
            gene_symbol = line[gene_symbol_index]
            protein_id = list(line[protein_id_index][2:-2].split('", "'))[0]
            monomer_ids_to_gene_symbols[protein_id] = gene_symbol

        return monomer_ids_to_gene_symbols

# add a gene common name column to priority proteases: 
priority_proteases['gene_id'] = priority_proteases['id'].apply(lambda x: get_gene_symbols_for_monomer_ids()[x])
priority_proteases



,id,protease_assignment,ClpP,Lon,HslV,Unexplained,gene_id
0,EG10156-MONOMER,ClpP only,1.000000,0.000000,0.000000,0.000000,clpA
1,MDLB-MONOMER,ClpP only,1.000000,0.000000,0.000000,0.000000,mdlB
2,EG10927-MONOMER,ClpP only,1.000000,0.000000,0.000000,0.000000,sbcC
3,EG10618-MONOMER,ClpP only,1.000000,0.000000,0.000000,0.000000,mukB
4,AERGLYC3PDEHYDROG-MONOMER,ClpP only,0.993731,0.000000,0.000000,0.000000,glpD
...,...,...,...,...,...,...,...
77,EG12690-MONOMER,"Additive: ClpP, Lon, HslV",0.277972,0.277364,0.378556,0.066108,rarA
78,PD03831,"Additive: ClpP, Lon",0.519958,0.403122,0.000000,0.076920,dnaA
79,AROK-MONOMER,"Additive: ClpP, Lon, HslV",0.319436,0.429511,0.167152,0.083901,aroK
80,G7263-MONOMER,"Additive: ClpP, Lon, HslV",0.183526,0.206054,0.520932,0.089488,murQ


In [ ]:
CLClimNE_HLs

# extract which proteins originally had 10 hour half lives

In [19]:
CLNE_NE = pd.read_csv('/Users/miagrahn/wcEcoli/models/ecoli/analysis/local_notebooks/C_limited_PDR_analyses_with_published_paper_data/previous_models/CLNE_files/CLNE_NE_rates_only.csv')

# convert the data to minute form
def convert_HL_data(dataframe):
    name_s = 'rate constant (s^1)' # name in seconds
    name_m = 'half life (min)' # name in mins
    dataframe.rename(columns={'Rate Constant': name_s}, inplace=True)
    dataframe[name_m] = dataframe[name_s] * 60 * (1/np.log(2))
    dataframe[name_m] = 1 / dataframe[name_m] # units of mins
    return dataframe

CLNE_NE = convert_HL_data(CLNE_NE)
CLNE_NE_10 = CLNE_NE[CLNE_NE['half life (min)'] > 599][['Protein ID']] # should be around 4296? 
CLNE_NE_2 = CLNE_NE[CLNE_NE['half life (min)'] < 600][['Protein ID']] # should be around 7

# find the relevant CLClimNE half lives: 
CLClimNE_was_NE_10 = CLClimNE_HLs[CLClimNE_HLs['monomer_id'].isin(CLNE_NE_10['Protein ID'])] # cool all 4296 should be there and they are!

# now remove the ones that are still N end rule: 
CLClimNE_Clim_10 = CLClimNE_was_NE_10[CLClimNE_was_NE_10['degradation_rate_source'] != 'N_end_rule'] # now 2668
CLClimNE_Clim_10


,monomer_id,degradation_rate(1/s),half_life_(min),degradation_rate_source
0,1-ACYLGLYCEROL-3-P-ACYLTRANSFER-MONOMER,0.000023,501.707893,Gupta_et_al_MS_2024
1,1-PFK-MONOMER,0.000015,766.256894,Gupta_et_al_MS_2024
3,2-ISOPROPYLMALATESYN-MONOMER,0.000042,276.924332,Gupta_et_al_MS_2024
4,2-OCTAPRENYL-METHOXY-BENZOQ-METH-MONOMER,0.000023,493.397050,Gupta_et_al_MS_2024
5,2-OCTAPRENYLPHENOL-HYDROX-MONOMER,0.000015,796.038829,Gupta_et_al_MS_2024
...,...,...,...,...
4302,YRBF-MONOMER,0.000014,823.833163,Gupta_et_al_MS_2024
4304,YTFQ-MONOMER,0.000015,790.418789,Gupta_et_al_MS_2024
4305,YTFR-MONOMER,0.000018,654.149344,Gupta_et_al_MS_2024
4307,ZNUA-MONOMER,0.000018,638.129701,Gupta_et_al_MS_2024


# find where the priority proteases fall

In [20]:
CLClimNE_Clim_10 = CLClimNE_Clim_10.merge(
    priority_proteases[['id', 'protease_assignment', 'gene_id']], left_on='monomer_id', right_on='id', how='left')
CLClimNE_Clim_10.drop(columns=['id'], inplace=True)
CLClimNE_Clim_10['protease_assignment'] = CLClimNE_Clim_10['protease_assignment'].fillna('None')


CLClimNE_Clim_10

,monomer_id,degradation_rate(1/s),half_life_(min),degradation_rate_source,protease_assignment,gene_id
0,1-ACYLGLYCEROL-3-P-ACYLTRANSFER-MONOMER,0.000023,501.707893,Gupta_et_al_MS_2024,None,NaN
1,1-PFK-MONOMER,0.000015,766.256894,Gupta_et_al_MS_2024,None,NaN
2,2-ISOPROPYLMALATESYN-MONOMER,0.000042,276.924332,Gupta_et_al_MS_2024,None,NaN
3,2-OCTAPRENYL-METHOXY-BENZOQ-METH-MONOMER,0.000023,493.397050,Gupta_et_al_MS_2024,None,NaN
4,2-OCTAPRENYLPHENOL-HYDROX-MONOMER,0.000015,796.038829,Gupta_et_al_MS_2024,None,NaN
...,...,...,...,...,...,...
2663,YRBF-MONOMER,0.000014,823.833163,Gupta_et_al_MS_2024,None,NaN
2664,YTFQ-MONOMER,0.000015,790.418789,Gupta_et_al_MS_2024,None,NaN
2665,YTFR-MONOMER,0.000018,654.149344,Gupta_et_al_MS_2024,None,NaN
2666,ZNUA-MONOMER,0.000018,638.129701,Gupta_et_al_MS_2024,None,NaN


In [21]:


CLClimNE_Clim_10 = CLClimNE_Clim_10.sort_values(by=["half_life_(min)"])
ordered_CLClimNE_Clim_10 = CLClimNE_Clim_10.copy()


# 
# CLClim0NE_df_full_protease_substrate_gene_symbols = []
# for gene_id in gene_ids:
#     if gene_id in priority_protease_data['Gene name'].values:
#         CLClim0NE_df_full_protease_substrate_gene_symbols.append(gene_id)
#         protease_row = priority_protease_data[priority_protease_data['Gene name'] == gene_id]
#         protease = protease_row['Protease assignment'].values[0]  
#         ordered_CLClim0NE_df_full.loc[ordered_CLClim0NE_df_full['gene symbol'] == gene_id, 'protease assignement'] = protease

# figure out which proteins were assigned to lon in the model: 
lon_proteins = ordered_CLClimNE_Clim_10[ordered_CLClimNE_Clim_10['protease_assignment'] == 'Lon only']
lon_proteins

# figure out which proteins were assigned to clp in the model:
clp_proteins = ordered_CLClimNE_Clim_10[ordered_CLClimNE_Clim_10['protease_assignment'] == 'ClpP only']

hslv_proteins = ordered_CLClimNE_Clim_10[ordered_CLClimNE_Clim_10['protease_assignment'] == 'HslV only']

# figure out which proteins are additive in the model: 
additive_proteins_LC = ordered_CLClimNE_Clim_10[ordered_CLClimNE_Clim_10['protease_assignment'] == 'Additive: ClpP, Lon']

# figure out which proteins are additive in the model: 
additive_proteins_LCH = ordered_CLClimNE_Clim_10[ordered_CLClimNE_Clim_10['protease_assignment'] == 'Additive: ClpP, Lon, HslV']

# figure out which proteins were not assigned to a protease in the model:
no_protease_proteins = ordered_CLClimNE_Clim_10[ordered_CLClimNE_Clim_10['protease_assignment'] == 'None']
print()

# todo: note that these numbers will not add up to those in the graph below bc some proteins in the CLClim0NE model are not in the CLNE model 
#298 total 
print("Lon Proteins (" +str(len(lon_proteins))+"): ", list(lon_proteins['gene_id'])) #6
print("ClpP Proteins (" +str(len(clp_proteins))+"): ", list(clp_proteins['gene_id'])) #40
print("HslV Proteins (" +str(len(hslv_proteins))+"): ", list(hslv_proteins['gene_id'])) #1
print("Additive (Lon + ClpP) Proteins (" +str(len(additive_proteins_LC))+"): ", list(additive_proteins_LC['gene_id'])) #33
print("Additive (Lon + ClpP + HslV) Proteins (" +str(len(additive_proteins_LCH))+"): ", list(additive_proteins_LCH['gene_id'])) #33

# figure out slices of the data:
ordered_CLClimNE_Clim_10_copy = ordered_CLClimNE_Clim_10.copy()
# make a copy of the ordered_df data:
CLClimNE_full_log_data_with_Lon_proteases = ordered_CLClimNE_Clim_10_copy.copy()
CLClimNE_full_log_data_with_ClpP_proteases = ordered_CLClimNE_Clim_10_copy.copy()
CLClimNE_full_log_data_with_HslV_proteases = ordered_CLClimNE_Clim_10_copy.copy()
CLClimNE_full_log_data_with_Additive_LC_proteases = ordered_CLClimNE_Clim_10_copy.copy()
CLClimNE_full_log_data_with_Additive_LCH_proteases = ordered_CLClimNE_Clim_10_copy.copy()
CLClimNE_full_log_data_with_no_proteases = ordered_CLClimNE_Clim_10_copy.copy()

# remove the proteins that were not assigned to the protease in the model:
CLClimNE_full_log_data_with_Lon_proteases = CLClimNE_full_log_data_with_Lon_proteases[CLClimNE_full_log_data_with_Lon_proteases['protease_assignment'] == 'Lon only']
CLClimNE_full_log_data_with_ClpP_proteases = CLClimNE_full_log_data_with_ClpP_proteases[CLClimNE_full_log_data_with_ClpP_proteases['protease_assignment'] == 'ClpP only']
CLClimNE_full_log_data_with_HslV_proteases = CLClimNE_full_log_data_with_HslV_proteases[CLClimNE_full_log_data_with_HslV_proteases['protease_assignment'] == 'HslV only']
CLClimNE_full_log_data_with_Additive_LC_proteases = CLClimNE_full_log_data_with_Additive_LC_proteases[CLClimNE_full_log_data_with_Additive_LC_proteases['protease_assignment'] == 'Additive: ClpP, Lon']
CLClimNE_full_log_data_with_Additive_LCH_proteases = CLClimNE_full_log_data_with_Additive_LCH_proteases[CLClimNE_full_log_data_with_Additive_LCH_proteases['protease_assignment'] == 'Additive: ClpP, Lon, HslV']

priority_substrates_in_the_full_model = [CLClimNE_full_log_data_with_Lon_proteases, CLClimNE_full_log_data_with_ClpP_proteases, CLClimNE_full_log_data_with_HslV_proteases, CLClimNE_full_log_data_with_Additive_LC_proteases, CLClimNE_full_log_data_with_Additive_LCH_proteases]

# concatenate the dataframes:
priority_substrates_full_model_df = pd.concat(priority_substrates_in_the_full_model)
priority_substrates_full_model_df


Lon Proteins (6):  ['metR', 'nemA', 'ydcI', 'trxC', 'fhlA', 'rpoD']
ClpP Proteins (39):  ['phoH', 'rpoS', 'intA', 'clpA', 'patA', 'ribB', 'clpX', 'ftsZ', 'ydhQ', 'cysD', 'fadE', 'putA', 'nfo', 'dps', 'yfcZ', 'comR', 'exuR', 'glpD', 'dksA', 'lldD', 'obgE', 'parC', 'yheS', 'def', 'pcnB', 'yheO', 'rpoC', 'znuC', 'mukB', 'mdlB', 'uvrD', 'dnaB', 'mutS', 'rpsA', 'dnaK', 'sbcC', 'oxyR', 'rpoB', 'recA']
HslV Proteins (1):  ['uhpA']
Additive (Lon + ClpP) Proteins (3):  ['ibpA', 'grcA', 'dnaA']
Additive (Lon + ClpP + HslV) Proteins (32):  ['iscR', 'erpA', 'yibA', 'glaR', 'aroK', 'chaB', 'minE', 'yfhH', 'yaeP', 'proQ', 'ybaB', 'parE', 'yjgA', 'yiiQ', 'murQ', 'yihD', 'glnD', 'helD', 'mazF', 'ppiC', 'pspA', 'acpP', 'yggX', 'srmB', 'ycaR', 'rarA', 'ligA', 'rhlB', 'kbp', 'thiL', 'hprR', 'yiaU']


,monomer_id,degradation_rate(1/s),half_life_(min),degradation_rate_source,protease_assignment,gene_id
2339,PD03938,0.000146,79.151002,Gupta_et_al_MS_2024,Lon only,metR
1542,G6890-MONOMER,0.000054,214.018094,Gupta_et_al_MS_2024,Lon only,nemA
1476,G6737-MONOMER,0.000040,288.738238,Gupta_et_al_MS_2024,Lon only,ydcI
2435,RED-THIOREDOXIN2-MONOMER,0.000031,367.223275,Gupta_et_al_MS_2024,Lon only,trxC
2331,PD02936,0.000025,468.602732,Gupta_et_al_MS_2024,Lon only,fhlA
...,...,...,...,...,...,...
488,EG10844-MONOMER,0.000018,624.526702,Gupta_et_al_MS_2024,"Additive: ClpP, Lon, HslV",rhlB
1746,G7395-MONOMER,0.000018,635.846404,Gupta_et_al_MS_2024,"Additive: ClpP, Lon, HslV",kbp
2511,THI-P-KIN-MONOMER,0.000018,656.519837,Gupta_et_al_MS_2024,"Additive: ClpP, Lon, HslV",thiL
1623,G7057-MONOMER,0.000018,656.623355,Gupta_et_al_MS_2024,"Additive: ClpP, Lon, HslV",hprR


In [35]:
# make the snake graph with the protease data:
import plotly.graph_objects as go
import glob
import plotly.express as px


fig = go.Figure()

# generate the figure in the background first:
fig = px.scatter(ordered_CLClimNE_Clim_10, x=ordered_CLClimNE_Clim_10["monomer_id"], y=ordered_CLClimNE_Clim_10["half_life_(min)"])
fig.update_traces(marker_size=.01, opacity=.1)    

# figure out slices of the data:
priority_protease_df = ordered_CLClimNE_Clim_10.copy()
# make a copy of the ordered_df data:
CLClimNE_Clim0_log_data_with_Lon_proteases = priority_protease_df.copy()
CLClimNE_Clim0_log_data_with_ClpP_proteases = priority_protease_df.copy()
CLClimNE_Clim0_log_data_with_HslV_proteases = priority_protease_df.copy()
CLClimNE_Clim0_log_data_with_Additive_LC_proteases = priority_protease_df.copy()
CLClimNE_Clim0_log_data_with_Additive_LCH_proteases = priority_protease_df.copy()
CLClimNE_Clim0_log_data_with_no_protease_proteases = priority_protease_df.copy()

# remove the proteins that were not assigned to the protease in the model:
CLClimNE_log_data_with_Lon_proteases = CLClimNE_Clim0_log_data_with_Lon_proteases[CLClimNE_Clim0_log_data_with_Lon_proteases['protease_assignment'] == 'Lon only']
CLClimNE_log_data_with_ClpP_proteases = CLClimNE_Clim0_log_data_with_ClpP_proteases[CLClimNE_Clim0_log_data_with_ClpP_proteases['protease_assignment'] == 'ClpP only']
CLClimNE_log_data_with_HslV_proteases = CLClimNE_Clim0_log_data_with_HslV_proteases[CLClimNE_Clim0_log_data_with_HslV_proteases['protease_assignment'] == 'HslV only']
CLClimNE_Clim0_log_data_with_Additive_LC_proteases = CLClimNE_Clim0_log_data_with_Additive_LC_proteases[CLClimNE_Clim0_log_data_with_Additive_LC_proteases['protease_assignment'] == 'Additive: ClpP, Lon']
CLClimNE_Clim0_log_data_with_Additive_LCH_proteases = CLClimNE_Clim0_log_data_with_Additive_LCH_proteases[CLClimNE_Clim0_log_data_with_Additive_LCH_proteases['protease_assignment'] == 'Additive: ClpP, Lon, HslV']
CLClimNE_log_data_with_no_protease_proteases = CLClimNE_Clim0_log_data_with_no_protease_proteases[CLClimNE_Clim0_log_data_with_no_protease_proteases['protease_assignment'] == 'None']

# overlay the individual rates
fig.add_trace(go.Scatter(x=(CLClimNE_log_data_with_no_protease_proteases['monomer_id']), y=(CLClimNE_log_data_with_no_protease_proteases["half_life_(min)"]), mode='markers', name=("No assignment"+ ' '+ (str(np.shape(CLClimNE_log_data_with_no_protease_proteases['monomer_id'])[0]))), hovertext=CLClimNE_log_data_with_no_protease_proteases['monomer_id'], marker_size=2, marker=dict(color="lightseagreen", size=3, opacity=.2), yaxis="y1"))

fig.add_trace(go.Scatter(x=(CLClimNE_Clim0_log_data_with_Additive_LC_proteases['monomer_id']), y=(CLClimNE_Clim0_log_data_with_Additive_LC_proteases["half_life_(min)"]), mode='markers', name=("Additive: ClpP, Lon "+ (str(np.shape(CLClimNE_Clim0_log_data_with_Additive_LC_proteases['monomer_id'])[0]))), hovertext=CLClimNE_Clim0_log_data_with_Additive_LC_proteases['gene_id'], marker_size=4, marker=dict(color='mediumturquoise', size=3, opacity=1), yaxis="y1"))

fig.add_trace(go.Scatter(x=(CLClimNE_Clim0_log_data_with_Additive_LCH_proteases['monomer_id']), y=(CLClimNE_Clim0_log_data_with_Additive_LCH_proteases["half_life_(min)"]), mode='markers', name=("Additive: ClpP, Lon, HslV "+ (str(np.shape(CLClimNE_Clim0_log_data_with_Additive_LCH_proteases['monomer_id'])[0]))), hovertext=CLClimNE_Clim0_log_data_with_Additive_LCH_proteases['gene_id'], marker_size=4, marker=dict(color='orange', size=3, opacity=.6), yaxis="y1"))

fig.add_trace(go.Scatter(x=(CLClimNE_log_data_with_ClpP_proteases['monomer_id']), y=(CLClimNE_log_data_with_ClpP_proteases["half_life_(min)"]), mode='markers', name=("ClpP  "+ (str(np.shape(CLClimNE_log_data_with_ClpP_proteases['monomer_id'])[0]))), hovertext=CLClimNE_log_data_with_ClpP_proteases['gene_id'], marker_size=3, marker=dict(color='deeppink', size=3, opacity=.4), yaxis="y1"))

fig.add_trace(go.Scatter(x=(CLClimNE_log_data_with_Lon_proteases['monomer_id']), y=(CLClimNE_log_data_with_Lon_proteases["half_life_(min)"]), mode='markers', name=("Lon  "+ (str(np.shape(CLClimNE_log_data_with_Lon_proteases['monomer_id'])[0]))), hovertext=CLClimNE_log_data_with_Lon_proteases['gene_id'], marker_size=5, marker=dict(color="yellowgreen", size=3, opacity=.8), yaxis="y1"))

fig.add_trace(go.Scatter(x=(CLClimNE_log_data_with_HslV_proteases['monomer_id']), y=(CLClimNE_log_data_with_HslV_proteases["half_life_(min)"]), mode='markers', name=("HslV  "+ (str(np.shape(CLClimNE_log_data_with_HslV_proteases['monomer_id'])[0]))), hovertext=CLClimNE_log_data_with_HslV_proteases['gene_id'], marker_size=5, marker=dict(color="brown", size=3, opacity=.6), yaxis="y1"))
              

# Plot Specs
fig.update_layout(title=dict(text=f'The C-limited (Clim) half life values for the New WCM proteins assigned to Clim <br> half-lives that were assinged to a half-life of 10 hours in the original model ', font=dict(size=15)), xaxis_title='Protein ID', yaxis_title='Clim Half Life (min)')
fig.update_xaxes(visible=False)
fig.update_layout(autosize=False, width=700, height=600, showlegend=True)

# wont open in pycharm, so save as a html:
out_pth = "/Users/miagrahn/wcEcoli/models/ecoli/analysis/local_notebooks/C_limited_PDR_analyses_with_published_paper_data/Clim_sorting_combos/Clim3/Clim3d/" + current_sequence + "_ordered_with_protein_substrates_and_colored_with_random_high_protein.html"
out_pth = os.path.expanduser(out_pth) 
output_dir = os.path.dirname(out_pth)
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it does not exist
fig.write_html(out_pth, auto_open=True) # displays the figure on a default web brouser 


# same as above but I remove the one random protein first

In [34]:
# make the snake graph with the protease data:
import plotly.graph_objects as go
import glob
import plotly.express as px

# get rid of the random protein first: 
ordered_CLClimNE_Clim_10_filtered = ordered_CLClimNE_Clim_10[ordered_CLClimNE_Clim_10['monomer_id'] != 'EG12298-MONOMER']


# generate graph
fig = go.Figure()

# generate the figure in the background first:
fig = px.scatter(ordered_CLClimNE_Clim_10_filtered, x=ordered_CLClimNE_Clim_10_filtered["monomer_id"], y=ordered_CLClimNE_Clim_10_filtered["half_life_(min)"])
fig.update_traces(marker_size=.01, opacity=.01)    

# figure out slices of the data:
priority_protease_df = ordered_CLClimNE_Clim_10_filtered.copy()
# make a copy of the ordered_df data:
CLClimNE_Clim0_log_data_with_Lon_proteases = priority_protease_df.copy()
CLClimNE_Clim0_log_data_with_ClpP_proteases = priority_protease_df.copy()
CLClimNE_Clim0_log_data_with_HslV_proteases = priority_protease_df.copy()
CLClimNE_Clim0_log_data_with_Additive_LC_proteases = priority_protease_df.copy()
CLClimNE_Clim0_log_data_with_Additive_LCH_proteases = priority_protease_df.copy()
CLClimNE_Clim0_log_data_with_no_protease_proteases = priority_protease_df.copy()

# remove the proteins that were not assigned to the protease in the model:
CLClimNE_log_data_with_Lon_proteases = CLClimNE_Clim0_log_data_with_Lon_proteases[CLClimNE_Clim0_log_data_with_Lon_proteases['protease_assignment'] == 'Lon only']
CLClimNE_log_data_with_ClpP_proteases = CLClimNE_Clim0_log_data_with_ClpP_proteases[CLClimNE_Clim0_log_data_with_ClpP_proteases['protease_assignment'] == 'ClpP only']
CLClimNE_log_data_with_HslV_proteases = CLClimNE_Clim0_log_data_with_HslV_proteases[CLClimNE_Clim0_log_data_with_HslV_proteases['protease_assignment'] == 'HslV only']
CLClimNE_Clim0_log_data_with_Additive_LC_proteases = CLClimNE_Clim0_log_data_with_Additive_LC_proteases[CLClimNE_Clim0_log_data_with_Additive_LC_proteases['protease_assignment'] == 'Additive: ClpP, Lon']
CLClimNE_Clim0_log_data_with_Additive_LCH_proteases = CLClimNE_Clim0_log_data_with_Additive_LCH_proteases[CLClimNE_Clim0_log_data_with_Additive_LCH_proteases['protease_assignment'] == 'Additive: ClpP, Lon, HslV']
CLClimNE_log_data_with_no_protease_proteases = CLClimNE_Clim0_log_data_with_no_protease_proteases[CLClimNE_Clim0_log_data_with_no_protease_proteases['protease_assignment'] == 'None']

# overlay the individual rates
fig.add_trace(go.Scatter(x=(CLClimNE_log_data_with_no_protease_proteases['monomer_id']), y=(CLClimNE_log_data_with_no_protease_proteases["half_life_(min)"]), mode='markers', name=("No assignment"+ ' '+ (str(np.shape(CLClimNE_log_data_with_no_protease_proteases['monomer_id'])[0]))), hovertext=CLClimNE_log_data_with_no_protease_proteases['monomer_id'], marker_size=2, marker=dict(color="lightseagreen", size=3, opacity=.2), yaxis="y1"))

fig.add_trace(go.Scatter(x=(CLClimNE_Clim0_log_data_with_Additive_LC_proteases['monomer_id']), y=(CLClimNE_Clim0_log_data_with_Additive_LC_proteases["half_life_(min)"]), mode='markers', name=("Additive: ClpP, Lon "+ (str(np.shape(CLClimNE_Clim0_log_data_with_Additive_LC_proteases['monomer_id'])[0]))), hovertext=CLClimNE_Clim0_log_data_with_Additive_LC_proteases['gene_id'], marker_size=4, marker=dict(color='mediumturquoise', size=3, opacity=1), yaxis="y1"))

fig.add_trace(go.Scatter(x=(CLClimNE_Clim0_log_data_with_Additive_LCH_proteases['monomer_id']), y=(CLClimNE_Clim0_log_data_with_Additive_LCH_proteases["half_life_(min)"]), mode='markers', name=("Additive: ClpP, Lon, HslV "+ (str(np.shape(CLClimNE_Clim0_log_data_with_Additive_LCH_proteases['monomer_id'])[0]))), hovertext=CLClimNE_Clim0_log_data_with_Additive_LCH_proteases['gene_id'], marker_size=4, marker=dict(color='orange', size=3, opacity=.6), yaxis="y1"))

fig.add_trace(go.Scatter(x=(CLClimNE_log_data_with_ClpP_proteases['monomer_id']), y=(CLClimNE_log_data_with_ClpP_proteases["half_life_(min)"]), mode='markers', name=("ClpP  "+ (str(np.shape(CLClimNE_log_data_with_ClpP_proteases['monomer_id'])[0]))), hovertext=CLClimNE_log_data_with_ClpP_proteases['gene_id'], marker_size=3, marker=dict(color='deeppink', size=3, opacity=.4), yaxis="y1"))

fig.add_trace(go.Scatter(x=(CLClimNE_log_data_with_Lon_proteases['monomer_id']), y=(CLClimNE_log_data_with_Lon_proteases["half_life_(min)"]), mode='markers', name=("Lon  "+ (str(np.shape(CLClimNE_log_data_with_Lon_proteases['monomer_id'])[0]))), hovertext=CLClimNE_log_data_with_Lon_proteases['gene_id'], marker_size=5, marker=dict(color="yellowgreen", size=3, opacity=.8), yaxis="y1"))

fig.add_trace(go.Scatter(x=(CLClimNE_log_data_with_HslV_proteases['monomer_id']), y=(CLClimNE_log_data_with_HslV_proteases["half_life_(min)"]), mode='markers', name=("HslV  "+ (str(np.shape(CLClimNE_log_data_with_HslV_proteases['monomer_id'])[0]))), hovertext=CLClimNE_log_data_with_HslV_proteases['gene_id'], marker_size=5, marker=dict(color="brown", size=3, opacity=.6), yaxis="y1"))
              

# Plot Specs
fig.update_layout(title=dict(text=f'The C-limited (Clim) half life values for the New WCM proteins assigned to Clim <br> half-lives that were assinged to a half-life of 10 hours in the original model ', font=dict(size=15)), xaxis_title='Protein ID', yaxis_title='Clim Half Life (min)')
fig.update_xaxes(visible=False)
fig.update_layout(autosize=False, width=900, height=600, showlegend=True)

# wont open in pycharm, so save as a html:
out_pth = "/Users/miagrahn/wcEcoli/models/ecoli/analysis/local_notebooks/C_limited_PDR_analyses_with_published_paper_data/Clim_sorting_combos/Clim3/Clim3d/" + current_sequence + "_ordered_with_protein_substrates_and_colored.html"
out_pth = os.path.expanduser(out_pth) 
output_dir = os.path.dirname(out_pth)
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it does not exist
fig.write_html(out_pth, auto_open=True) # displays the figure on a default web brouser 
